In [1]:
# pip install osmnx

In [2]:
import osmnx as ox

In [3]:
# Code to retrieve street network and angles
G=ox.graph_from_address('Kyoto,Japan',network_type='drive',dist=5000)

ox.bearing.add_edge_bearings(G)

ox.plot_graph(G)

#turn graph into a geodataframe
edgeDF=ox.graph_to_gdfs(G,nodes=False)

#sample from this edgelist dataframe

In [4]:
len(edgeDF)
# 55458

In [5]:
edge_sample = edgeDF.sample(n=24000,replace=False,random_state=0)
edge_sample = edge_sample.reset_index()  

In [6]:
def get_firstpointlong(row):
    return row['geometry'].coords[0][0]
#     return np.sqrt((row["Eastings"]-piccadillycircus[0])**2 + (row["Northings"]-piccadillycircus[1])**2)
def get_firstpointlat(row):
    return row['geometry'].coords[0][1]

#new feature
edge_sample["long"] = edge_sample.apply(lambda row: get_firstpointlong(row), axis=1)
edge_sample["lat"] = edge_sample.apply(lambda row: get_firstpointlat(row), axis=1)

In [7]:
edge_sample.head()

In [8]:
# save dataframe as csv
edge_sample.to_csv('dataframe.csv', index = False)

In [9]:
edge_sample.columns

In [10]:
import csv
data = open('data.csv', 'w',newline='')
data_writer = csv.writer(data)
data_writer.writerow(['index','u','v','osmid','long','lat','bearing','geometry'])

data_backup = open('data_backup.csv', 'w',newline='')
data_backup_writer = csv.writer(data_backup)
data_backup_writer.writerow(['index','u','v','osmid','long','lat','bearing','geometry'])

In [11]:
# edge_sample(edge_sample.iloc[15906])
# edge_sample(edge_sample.iloc[15907])
# edge_sample(edge_sample.iloc[15908])
# edge_sample(edge_sample.iloc[15909])
# edge_sample(edge_sample.iloc[15910])

# GetStreet(1000000,edge_sample.iloc[0]['bearing'],
#           edge_sample.iloc[0]['lat'],edge_sample.iloc[0]['long'])

In [12]:
# # parameters
# # heading: 0 & 360 is North & 90 is East & 180 is South
# # fov: horizontal field of view of the image (default is 90)
# # smaller fov indicates higher level of zoom
# # pitch up or down angle of the camera (default is 0)
# # 90 is straight up & -90 is straight down

# #write to csv

import os
import urllib.parse
import urllib.request

key = "&key="+'APIKEY' #API KEY

def GetStreet(num,heading,lat,long):
    pic_base = "https://maps.googleapis.com/maps/api/streetview?size=640x640&source=outdoor&"
    pic_base = pic_base + "location=" + str(lat) + "," + str(long) 
    pic_base = pic_base + "&heading=" + str(heading)
    pic_base = pic_base + key #added url encoding
    fi = str(lat) + "_" + str(long) + ".jpg"  #add lat long to image
#     print(pic_base)
    fi = str(num) + ".jpg"
    urllib.request.urlretrieve(pic_base, os.path.join("images",fi))


n = 0
# for index, row in edge_sample.iloc[15914:].iterrows(): #iterate each row in dataframe
for index, row in edge_sample.iterrows():
    print(index)
    GetStreet(n,row['bearing'],row['lat'],row['long'])
    n+=1
    data_writer.writerow([index,row['u'],row['v'],row['osmid'],row['long'],row['lat'],row['bearing'],row['geometry']])
    data_backup_writer.writerow([index,row['u'],row['v'],row['osmid'],row['long'],row['lat'],row['bearing'],row['geometry']])
data.close()
data_backup.close()
#     print(row['c1'], row['c2'])    
    
# for i in range(len(Tests)):
#     GetStreet(str(i),Tests[i])